# Offline RL for Loan Approval (Contextual Bandit)

This notebook frames loan approval as an **offline RL / contextual bandit** problem and trains a simple offline agent using **Fitted Q / supervised regression**. It treats each loan application as a single-step decision (approve or deny). The notebook is intentionally standalone: it loads a CSV, preprocesses features, engineers rewards according to your specification, trains per-action regressors to estimate Q(s,a), derives a policy `pi(s)=argmax_a Q(s,a)`, evaluates on a holdout test set, and saves the trained models.

**Reward design (per your spec):**
- If action == 0 (Deny): reward = 0
- If action == 1 (Approve) and loan was fully paid: reward = + (loan_amnt * int_rate)
- If action == 1 (Approve) and loan defaulted: reward = - loan_amnt

---

**How to use:**
1. Upload a CSV dataset containing loan records (common columns used: `loan_amnt`, `int_rate`, `loan_status` or `fully_paid` or `is_default`).
2. Update the `COLUMN_MAPPING` cell below if your column names differ.
3. Run all cells. The notebook will produce `outputs/` with models and metrics.

This is intentionally simple and reproducible; for production-grade offline RL you would use specialized libraries (CQL, BCQ, CRR, etc.).


In [ ]:
# Imports & helper functions
import pandas as pd
import numpy as np
import os, joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

os.makedirs('outputs', exist_ok=True)

def infer_boolean_fully_paid(df):
    """Try to infer a boolean fully-paid column from common loan_status values."""
    # Common values: 'Fully Paid', 'Charged Off', 'Default', 'Current', 'Late'
    if 'loan_status' in df.columns:
        col = df['loan_status'].astype(str).str.lower()
        fully = col.isin(['fully paid', 'fully_paid', 'paid', 'paid off']) | col.str.contains('fully paid')
        default = col.isin(['charged off', 'default', 'charged_off']) | col.str.contains('charged')
        res = pd.Series(np.nan, index=df.index)
        res[fully] = True
        res[default] = False
        return res
    # try other columns
    for candidate in ['fully_paid', 'is_default', 'default_flag', 'repaid']:
        if candidate in df.columns:
            col = df[candidate]
            return col.astype('bool')
    return None

def compute_reward(row, loan_amnt_col='loan_amnt', int_rate_col='int_rate', fully_paid_col='fully_paid'):
    """Compute reward for a single row using the provided column names and the problem's reward design."""
    loan_amnt = float(row[loan_amnt_col])
    int_rate = float(row[int_rate_col])
    fully = row[fully_paid_col]
    if isinstance(int_rate, str) and '%' in int_rate:
        int_rate = float(int_rate.replace('%','')) / 100.0
    if int_rate > 1.0:
        int_rate = int_rate / 100.0
    return {'approve_reward': (loan_amnt * int_rate) if fully else (-loan_amnt),
            'deny_reward': 0.0}


In [ ]:
# === USER: Set your CSV path and column mapping here ===
CSV_PATH = 'loan_data.csv'  # <-- change to your uploaded CSV filename
COLUMN_MAPPING = {
    'loan_amnt': 'loan_amnt',   # principal
    'int_rate': 'int_rate',     # interest rate (float or percent string)
    'loan_status': 'loan_status' # column used to infer fully paid vs default
}
fully_paid_col_override = None  # e.g., 'fully_paid' or None to infer from loan_status

import os
if not os.path.exists(CSV_PATH):
    print(f"Warning: CSV_PATH '{CSV_PATH}' not found in working directory. Upload your CSV or update the path.") 
else:
    print('CSV found:', CSV_PATH)


In [ ]:
# Load data, infer outcome (fully paid), and compute rewards
df = pd.read_csv(CSV_PATH)
print('Loaded rows:', len(df))
print('Columns:', list(df.columns)[:50])

if fully_paid_col_override and fully_paid_col_override in df.columns:
    df['fully_paid'] = df[fully_paid_col_override].astype(bool)
else:
    inferred = infer_boolean_fully_paid(df)
    if inferred is not None:
        df['fully_paid'] = inferred
    else:
        raise ValueError('Could not infer fully_paid outcome from dataset. Please set fully_paid_col_override to the appropriate column name.')

missing_outcome = df['fully_paid'].isna().sum()
if missing_outcome > 0:
    print(f'Warning: {missing_outcome} rows have unknown outcome; dropping them.')
    df = df[~df['fully_paid'].isna()].copy()

for key, col in COLUMN_MAPPING.items():
    if col not in df.columns:
        raise ValueError(f"Expected column '{col}' for mapping key '{key}' not found in CSV. Update COLUMN_MAPPING.")

loan_amnt_col = COLUMN_MAPPING['loan_amnt']
int_rate_col = COLUMN_MAPPING['int_rate']

rewards = df.apply(lambda r: compute_reward(r, loan_amnt_col=loan_amnt_col, int_rate_col=int_rate_col, fully_paid_col='fully_paid'), axis=1)
df['approve_reward'] = [x['approve_reward'] for x in rewards]
df['deny_reward'] = 0.0

observed_action_col = None  # e.g., 'was_approved' or 'policy_action' - set to None if unknown
if observed_action_col and observed_action_col in df.columns:
    df['observed_action'] = df[observed_action_col].astype(int)
else:
    df['observed_action'] = np.nan

print('Prepared dataset with computed rewards. Sample:')
display(df.head().T.iloc[:20])


In [ ]:
# Identify features automatically: drop target and outcome columns
exclude_cols = ['approve_reward', 'deny_reward', 'fully_paid', 'observed_action']
feature_cols = [c for c in df.columns if c not in exclude_cols and c not in [loan_amnt_col, int_rate_col]]
feature_cols = [loan_amnt_col, int_rate_col] + [c for c in feature_cols if c not in (loan_amnt_col, int_rate_col)]
print('Using feature columns (first 30):', feature_cols[:30])

num_cols = df[feature_cols].select_dtypes(include=['int64','float64','int32','float32']).columns.tolist()
cat_cols = [c for c in feature_cols if c not in num_cols]

print('Numeric cols:', num_cols)
print('Categorical cols:', cat_cols)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ], remainder='drop')

X = df[feature_cols].copy()
y_approve = df['approve_reward'].values
y_deny = df['deny_reward'].values

X_trans = preprocessor.fit_transform(X)
print('Transformed feature shape:', X_trans.shape)


In [ ]:
# Train regressors to estimate Q(s,a) for each action (0: deny, 1: approve)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

X_train, X_test, y1_train, y1_test, y0_train, y0_test = train_test_split(X_trans, y_approve, y_deny, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

model_approve = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_deny = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

model_approve.fit(X_train, y1_train)
model_deny.fit(X_train, y0_train)

joblib.dump(model_approve, 'outputs/model_approve.pkl')
joblib.dump(model_deny, 'outputs/model_deny.pkl')
joblib.dump(preprocessor, 'outputs/preprocessor.pkl')

print('Models trained and saved to outputs/.')

In [ ]:
# Derive deterministic policy pi(s) = argmax_a Q_hat(s,a) and evaluate on test set
from sklearn.model_selection import train_test_split as tts2
q1_test = model_approve.predict(X_test)
q0_test = model_deny.predict(X_test)
policy_actions = (q1_test > q0_test).astype(int)

indices = np.arange(X_trans.shape[0])
_, test_idx = tts2(indices, test_size=0.2, random_state=42)
df_test = df.iloc[test_idx].copy().reset_index(drop=True)

df_test['policy_action'] = policy_actions.tolist()
df_test['policy_reward'] = df_test.apply(lambda r: r['approve_reward'] if r['policy_action']==1 else 0.0, axis=1)

if df_test['observed_action'].notna().any():
    df_test['behavior_reward'] = df_test.apply(lambda r: r['approve_reward'] if r['observed_action']==1 else 0.0, axis=1)
else:
    df_test['behavior_reward'] = np.nan

policy_total_reward = df_test['policy_reward'].sum()
policy_avg_reward = df_test['policy_reward'].mean()
behavior_total_reward = df_test['behavior_reward'].sum() if df_test['behavior_reward'].notna().any() else np.nan

print(f"Policy total reward on test set: {policy_total_reward:.2f}")
print(f"Policy average reward per-applicant: {policy_avg_reward:.4f}")
if not np.isnan(behavior_total_reward):
    print(f"Observed behavior total reward on test set: {behavior_total_reward:.2f}")
print('Policy approval rate (fraction approved):', df_test['policy_action'].mean())

approved = df_test[df_test['policy_action']==1]
if len(approved) > 0:
    frac_fully = approved['fully_paid'].mean()
    print('Fraction of policy-approved applicants who truly repaid:', frac_fully)
else:
    print('Policy approved zero applicants on test set.')

In [ ]:
# Save a quick policy wrapper that loads preprocessor and models and returns actions for a DataFrame of applicants
policy_code = '''import joblib, numpy as np, pandas as pd
pre = joblib.load('outputs/preprocessor.pkl')
m1 = joblib.load('outputs/model_approve.pkl')
m0 = joblib.load('outputs/model_deny.pkl')
def policy_from_df(df, feature_cols):
    X = df[feature_cols].copy()
    X_trans = pre.transform(X)
    q1 = m1.predict(X_trans)
    q0 = m0.predict(X_trans)
    actions = (q1 > q0).astype(int)
    return actions
'''
with open('outputs/policy_wrapper.py', 'w') as f:
    f.write(policy_code)
print('Saved outputs/policy_wrapper.py')